# A Visual Exploration of Movie Profit Trends
- Module 1 Project by Charlotte Basch with Rachel Beery
- https://github.com/charlottebasch/A-Visual-Exploration-of-Movie-Profit-Trends

#  Introduction

&nbsp; &nbsp; &nbsp;Before the events of the Coronavirus pandemic, the film industry was a tremendously profitable business. In <a href="https://www.billboard.com/articles/news/8547827/2019-global-box-office-revenue-hit-record-425b-despite-4-percent-dip-in-us#:~:text=Global%20box%20office%20revenue%20hit%20a%20record%20%2442.5%20billion%20in,%2411.88%20billion%2C%20according%20to%20Comscore." target="_blank">2019</a>, global box office revenue reached 42.5 billion dollars. In this project we explored data about movie production from 2010 to 2018 using pandas to work with datasets and make visualizations. This project contains data we were given for the module one project in order to make recommendations to Microsoft if they were considering going into the movie business. We were provided with datasets from IMDB, TMDb, and The Numbers. These datasets provided information on movie titles, years, runtime, genres, budgets, profits, and other related variables. The data also contained audience ratings. We focused on three questions about what factors are related to film grosses. 

## Questions:
- Do IMDB user ratings relate to profit or budget?
- What genres are the most profitable?
- Does runtime affect movie gross?

#  Data Cleaning

We began by combining the datasets in a way that allowed us to answer the questions we intended to explore. 

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
#Import data from imdb, tmdb, and tn
pd.set_option("display.max_rows", None, "display.max_columns", None)
imdb_title_basics = pd.read_csv(r"zippedData\imdb.title.basics.csv.gz")
imdb_title_ratings = pd.read_csv(r"zippedData\imdb.title.ratings.csv.gz")
tmdb_movies = pd.read_csv(r"zippedData\tmdb.movies.csv.gz")
tn_movie_budgets = pd.read_csv(r"zippedData\tn.movie_budgets.csv.gz")

## IMDB

In [ ]:
display(imdb_title_basics.head(10))

In [ ]:
#Fill the missing runtime values with the median to preserve the distriibution of runtimes 
imdb_title_basics["runtime_minutes"] = imdb_title_basics["runtime_minutes"].fillna(imdb_title_basics["runtime_minutes"].median())

#Fill missing values with 'missing' to indicate that they are gone.
imdb_title_basics["genres"] = imdb_title_basics["genres"].fillna("Missing")
imdb_title_basics["original_title"] = imdb_title_basics["original_title"].fillna("Missing")

display(imdb_title_ratings.head(10))

In [ ]:
#Join together titles and user ratings on imdb movie id
titles_with_ratings = imdb_title_basics.join(imdb_title_ratings.set_index("tconst"), on="tconst", how="right")

display(titles_with_ratings.head())

## TMDb and The Numbers

In [ ]:
#Get rid of unamed column 
tmdb_movies = tmdb_movies.drop(columns=["Unnamed: 0"])

#Categorize movies by year instead of date
tmdb_movies["release_date"] = tmdb_movies["release_date"].str[:4]
tn_movie_budgets["release_date"] = tn_movie_budgets["release_date"].str[-4:]

#Rename the title column to aid merging 
tmdb_movies.rename(columns={"title": "movie"}, inplace=True)

display(tmdb_movies.head())
display(tn_movie_budgets.head())

In [ ]:
#Join together tmdb genre information with movie budgets based on movie name
budgets_and_ratings = tn_movie_budgets.merge(tmdb_movies, on="movie", how="inner")

#Drop repeated columns
budgets_and_ratings = budgets_and_ratings.drop(columns=["id_x", "id_y", "genre_ids"])

#Make sure movies are correct by matching release year 
budgets_and_ratings = budgets_and_ratings.loc[budgets_and_ratings['release_date_x'] == budgets_and_ratings['release_date_y']]

In [ ]:
#Now that we've verified the movies are correct, drop the repeated year column
budgets_and_ratings = budgets_and_ratings.drop(columns=["release_date_y"])

#Rename the remaining year column
budgets_and_ratings.rename(columns={"release_date_x": "year"}, inplace=True)

display(budgets_and_ratings.head())

In [ ]:
#Rename column to aid with merging
titles_with_ratings.rename(columns={"original_title": "movie"}, inplace=True)

#Merge budget and genre information with imdb user ratings based on movie title
budgets_and_ratings_with_imdb = budgets_and_ratings.merge(titles_with_ratings, on="movie", how="inner")

#Drop unnecessary columns
budgets_and_ratings_with_imdb = budgets_and_ratings_with_imdb.drop(columns=["tconst"])
budgets_and_ratings_with_imdb.drop_duplicates(keep="first", inplace=True) 

display(budgets_and_ratings_with_imdb.head())

In [ ]:
#Make the year a number so it can be compared
budgets_and_ratings_with_imdb = budgets_and_ratings_with_imdb.astype({'year': 'float64'})

#Make sure the years match on movies with the same title
budgets_and_ratings_with_imdb = budgets_and_ratings_with_imdb.loc[budgets_and_ratings_with_imdb['year'] == budgets_and_ratings_with_imdb['start_year']]

#Get rid of all irrelevant columns
budgets_and_ratings_with_imdb = budgets_and_ratings_with_imdb.drop(columns=["start_year", "original_language", 
                                                                            "popularity", "vote_average", "vote_count",
                                                                            "original_title", "primary_title"])

#Make sure only unqiue movies are in the dataset
movies = budgets_and_ratings_with_imdb.drop_duplicates(subset=["year", "movie"], keep="first")

display(movies.head())

## Clean the Final Dataset

In order to make visual exploration of our data easier, we must modify the form of some of our final variables.
- Our monetary variables are not in the correct data form and since we are working with such larger amounts of money, our visuals will be more interpretable if we scale down the numbers.
- Right now our movie genres are in a list, but it will be much simpler for each movie to be classified by whether it falls in each genre given that movies can be examples of more than one genre. 

In [ ]:
#Change budget, gross, and year data into the correct data form and convert the 
#monetary values them to a measure of 'millions of dollars'
movies = movies.astype({"year": "int32"})
cols = ["production_budget", "domestic_gross", "worldwide_gross"]
for col in cols:
    movies[col] = movies[col].str[1:]
    movies[col] = movies[col].replace(',','', regex=True)
    movies = movies.astype({col: "float64"})
    movies[col] = movies[col]/1000000

In [ ]:
#Create a list of all genres and restructure 
values = movies["genres"].value_counts()
values_list = list(values.index)
genres_list = []
for entry in values_list:
    genres = entry.split(",")
    for genre in genres:
        if genre not in genres_list:
            genres_list.append(genre)

#Create new columns to indicate which genres each movie falls into 
for genre in genres_list:
    movies[genre] = movies["genres"].str.contains(genre)

In [ ]:
display(movies.head())

#  Questions

## Do IMDB user ratings relate to profit or budget?

While there are certainly benefits to producing a movie people generally enjoy, it is unclear if opt-in online ratings from websites such as IMDB offer any kind of actionable insight into making movies more people will enjoy and therefore pay to see. We looked to see there was any correlation between average IMDB ratings, the number of IMDB ratings for a movie, and its budget or profits. Next we looked at whether there was a graphical relationship between the worldwide gross and average IMDB rating. 

### Correlation

In [ ]:
#Get correlations for imdb ratings data and production data
movies_subset = movies[["production_budget", "domestic_gross", 
                "worldwide_gross", "averagerating", "numvotes"]]
corr_subset = movies_subset.copy()
corr = corr_subset.corr()

In [ ]:
#Create a heatmap to display the correlations 
fig, ax = plt.subplots(figsize=(8,8))
mask = np.triu(np.ones_like(corr, dtype=np.bool))
labels = ["Production Budget", "Domestic Gross", "Worldwide Gross", "Average IMDB User Rating", 
          "Number of IMDB Ratings"]
sns.heatmap(corr, mask=mask, square=True, annot=True, cmap="YlGnBu", xticklabels=labels, yticklabels=labels)
ax.patch.set_edgecolor('black')  
ax.patch.set_linewidth('1')
ax.set_title("Movie Production and Audience Rating Correlations", fontsize=15, fontfamily="serif");

- As can be reasonably expected, the only strong correlations were between the financial variables.
- There was a moderate correlation between the number of ratings a movie had and its gross. This makes sense as more people saw these movies.
- There is also a moderate correlation between the number of reviews and the budget. Given that a movie's budget is fairly strongly related to the gross, this is again unsurprising.
- The correlations between the average rating and both gross and budget is fairly weak, indicating that these reviews are perhaps not a good predictor of gross. 

### Scatterplot

In [ ]:
# answering the question "Does IMDb rating predict profitability?" via scatterplot"

figimdb = px.scatter(movies, x="averagerating", y="worldwide_gross", trendline="lowess",
                    labels={
                        "averagerating": "Average IMDb Rating (0-10)",
                        "worldbwide_gross": "Worldwide Gross Profits"
                        
                    },
                    title="Worldwide Grossing by Average IMDb Rating")
figimdb.update_layout(
    title={
        'y': 0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    })
figimdb.show()

- While there is a slight positive trend in this data, the wide spread of points does not indicate a clear relationship.
- Most points are fairly low regardless of rating, with a smaller number of movies at the higher ratings with large grosses may explain the small positive relationship.

### Conclusion

- Online rating systems such as IMDB likely do not accurately reflect the viability of a movie.
- This is a small sample of people who chose to rate movies and their enjoyment may not reflect the spending preferences of the general public. 

## What genres are the most profitable?

Genre is undoubtedly an important factor in a movie's profitability. When deciding what type of movies to make, it may be valuable to know which genres tend to make the most money.   

- First we must get the totals and medians for each genre for domestic gross, international gross, and budget. 
- We choose the median to avoid the outsized influence of movie series that generate great sums of money but are not necessarily representative of the genre's earning potential. 

In [ ]:
#Get the totals and averages for the financial variables
world_values = [movies[movies[genre] == True]["worldwide_gross"].sum() for genre in genres_list]
dom_values = [movies[movies[genre] == True]["domestic_gross"].sum() for genre in genres_list]
cost_values = [movies[movies[genre] == True]["production_budget"].sum() for genre in genres_list]
cost_medians = [movies[movies[genre] == True]["production_budget"].median() for genre in genres_list]
dom_medians = [movies[movies[genre] == True]["domestic_gross"].median() for genre in genres_list]
world_medians = [movies[movies[genre] == True]["worldwide_gross"].median() for genre in genres_list]

In [ ]:
#Create a new dataframe with the total aggregated data and one with the average data
grosses = pd.DataFrame(list(zip(genres_list, cost_values, dom_values, world_values)), 
                       columns=["Genre", "Production Budget", "Domestic Gross", "Worldwide Gross"])
grosses = grosses.sort_values(by="Domestic Gross", ascending=False)

medians = pd.DataFrame(list(zip(genres_list, cost_medians, dom_medians, world_medians)), 
                       columns=["Genre", "Production Budget", "Domestic Gross", "Worldwide Gross"])
medians = medians.sort_values(by="Worldwide Gross", ascending=False)

#View the new dataframes
print("Total Gross by Genre")
display(grosses.head())
print("Median gross by genre")
display(medians.head())

## Median Gross by Genre

- We start by looking at the median domestic and international gross for each genre. 

In [ ]:
#Create a horizontal barplot of average domestic and worldwide gross by genre
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize = (15,15))
sns.set_color_codes("pastel")
sns.barplot(x="Worldwide Gross", y="Genre", data=medians, label="Worldwide", color="b")
sns.set_color_codes("muted")
sns.barplot(x="Domestic Gross", y="Genre", data=medians, label="Domestic", color="b")
ax.legend(ncol=2, loc="lower right")
ax.set(ylabel="Genre", xlabel="Median Gross (Millions of Dollars)", title="Movie Gross by Genre (2010-2018)")
sns.despine(left=True, bottom=True)
plt.show()

- Animation, adventure, and sci-fi movies have the highest median gross both domestically and internationally indicating that these genres retain great popularity across culture. 

## Yearly Gross

- While the visual above is very compelling, it is important to look at each genre over time to make sure these top genres are not on the downswing. 
- Therefore we will get the total gross for each 

In [ ]:
#Get a list of all years
years = list(movies["year"].value_counts().index)

#Sort the years
years.sort()

#Generate the total by genre for each year
genre_cols = []
year_cols = []
yearly_total_cols = []
for genre in genres_list:
    for year in years:
        #We 
        yearly_total = movies[(movies[genre] == True) & (movies["year"] == year)]["worldwide_gross"].sum()
        genre_cols.append(genre)
        year_cols.append(year)
        yearly_total_cols.append(yearly_total)

#Create a new dataframe based on the yearly data
yearly_gross = pd.DataFrame(list(zip(genre_cols, year_cols, yearly_total_cols)), 
                                columns=["Genre", "Year", "Yearly Gross"])

display(yearly_gross.head())

In [ ]:
#Create a plot for each genre showing yearly gross for the stated years
g = sns.lmplot(x="Year", y="Yearly Gross", col="Genre", hue="Genre", data=yearly_gross,
           col_wrap=7, palette="muted", height=3)
fig = g.fig
axes = fig.get_axes()
fig.suptitle("Yearly Gross by Genre In Millions (2010-2018)", y=1.05, fontsize=20);

- Action, adventure, and sci-fi are the genres most on the rise, possibly owing to the rise of franchises such as the Marvel Cinematic Universe and the Hunger Games during this time period.  

## Number of Movies Made by Genre

- We can also examine the cost and profit of genres in relation to how many movies of each type were made. 

In [ ]:
#Count the number of movies of each type
counts = []
for genre in genres_list:
    counts.append(movies[genre].value_counts().loc[True])

#Add the counts to the grosses dataframe
grosses["Number of Movies"] = counts

display(grosses.head())

In [ ]:
#Create a scatterplot compairing budget and worldwide gross by genre, displaying 
#the number of movies of each genre that were made
fig = px.scatter(grosses, x="Production Budget", y="Worldwide Gross",
                 size="Number of Movies", color="Genre",
                 hover_name="Genre", log_x=True, size_max=60)
fig.update_layout(
    title={
        'text': "Number of Movies per Genre",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Production Budget (Millions of Dollars)",
    yaxis_title="Worldwide Gross (Millions of Dollars)")
fig.show()

## Conclusion

 - Making action, adventure, animation, and sci-fi movies increases the likelihood of making large profits. However multiple other genres are also continuing to bring in impressive box offices. 
 - Certain genres, such as sci-fi can make similar profits and use similar budgets as other genres, such as animation, while making fewer movies. 
 - While we cannot choose one strategy as superior, it is worth considering whether money should be put towards a few films or many. 

## Does runtime affect movie gross?

In [ ]:
#Create a scatterplot for runtime versus domestic and international gross
plt.figure(figsize=(10, 8))
fig, (ax1, ax2)  = plt.subplots(1, 2, sharey=True)
ax1.scatter(movies["runtime_minutes"], movies["domestic_gross"])
ax1.set(title="Domestic Gross", xlabel="Runtime (Minutes)", ylabel="Domestic Gross (Millions of Dollars)")
ax2.scatter(movies["runtime_minutes"], movies["worldwide_gross"])
ax1.set(title="Domestic Gross", xlabel="Runtime (Minutes)", ylabel="Gross (Millions of Dollars)")
ax2.set(title="Worldwide Gross", xlabel= "Runtime (Minutes)")
fig.suptitle("Gross by Runtime", y=1.05)

Recommendation: Once runtime hits approximately 75 minutes, it is not an important predictor of profit.

# Conclusion

- Online rating should not be used when making decisions about budget or to predict profit. 
- Action, animation, sci-fi, and adventure movies are the most profitable genres. However smaller grossing genres still make a lot of profit. Additionally, many of these genres overlap so the profits observed are not completely additive. 
- Runtime is not related to movie gross unless it is unusually short. 

The movie business continues to expand and by examining the data and isolating the relevant factors, Microsoft would be able to maximize their profits in this kind of endeavor. 